# Cleaning Data/EDA

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score, mean_squared_error


%matplotlib inline

In [ ]:
train = pd.read_csv('../datasets/train.csv')#,index_col=0)
test = pd.read_csv('../datasets/test.csv')#, index_col=0)

In [ ]:
train.head()

In [ ]:
def noval(col):
    print((col).isnull().sum())

In [ ]:
noval(train['Lot Frontage'])

In [ ]:
train['Lot Frontage'].fillna(0, inplace=True)
#filling NA's in this numerical column with 0

In [ ]:
noval(train['Lot Area'])

In [ ]:
train['Alley'].fillna('No Access', inplace=True)
train['Alley'].value_counts()
#data description states "NA" values indicate no alley access, so I replace those blanks with "No Access"

In [ ]:
train['Condition 2'].value_counts()

In [ ]:
train['Conditions']= train['Condition 1'].str.cat(train['Condition 2'], sep=",")
#Condition 1 and Condition 2 deal with the same values; I combined into one column (separating vals with a comma) for ease/to prevent future high multicollinearity

In [ ]:
train.drop(['Condition 1', 'Condition 2'], axis=1, inplace=True)

In [ ]:
noval(train['Mas Vnr Type'])

In [ ]:
noval(train['Mas Vnr Area'])

In [ ]:
#making sure the 22 null rows match up with respective columns;
#rather than get rid of them, I'll replace them with relevant assumed values
(train[train['Mas Vnr Type'].isnull()]).index & (train[train['Mas Vnr Area'].isnull()]).index

In [ ]:
train['Mas Vnr Type'].fillna('None', inplace=True)
train['Mas Vnr Area'].fillna(0, inplace=True)

In [ ]:
(train[train['Bsmt Cond'].isnull()].index) & (train[train['Bsmt Qual'].isnull()].index) & (train[train['Bsmt Exposure'].isnull()].index)

In [ ]:
#there are more null vals in 'Bsmt Exposure'
#after identifying indexes that do not match; replacing those appropriately
(train[train['Bsmt Exposure'].isnull()].index)

In [ ]:
train.loc[1997, 'Bsmt Exposure']='No'
train.loc[1547, 'Bsmt Exposure']='No'
train.loc[1456, 'Bsmt Exposure']='No'

In [ ]:
#'NB' for no basement
train['Bsmt Qual'].fillna('NB', inplace=True)
train['Bsmt Cond'].fillna('NB', inplace=True)
train['Bsmt Exposure'].fillna('NB', inplace=True)

In [ ]:
train['BsmtFin Type 1'].fillna('NB', inplace=True)

In [ ]:
train['BsmtFin SF 1'].fillna(0, inplace=True)

In [ ]:
train['BsmtFin Type 2'].fillna('NB', inplace=True)

In [ ]:
train['BsmtFin SF 2'].fillna(0, inplace=True)

In [ ]:
train['Bsmt Unf SF'].fillna(0, inplace=True)

In [ ]:
train['BsmtFin SF'] = train['BsmtFin SF 1'] + train['BsmtFin SF 2']

In [ ]:
#rename to match pattern of other basement names/easier to call
train.rename(columns ={'Total Bsmt SF': 'Bsmt Total SF'}, inplace=True)
train['Bsmt Total SF'].fillna(0, inplace=True)

In [ ]:
train[train['Bsmt Full Bath'].isnull()].index

In [ ]:
train[train['Bsmt Half Bath'].isnull()].index

In [ ]:
train['Bsmt Half Bath'].fillna(0, inplace=True)
train['Bsmt Full Bath'].fillna(0, inplace=True)

In [ ]:
train['Bsmt Total Baths'] = train['Bsmt Full Bath'] + train['Bsmt Half Bath']

In [ ]:
train.drop('Bsmt Full Bath',axis=1, inplace=True)
train.drop('Bsmt Half Bath', axis=1,inplace=True)

In [ ]:
train['Total Baths'] = train['Full Bath'] + train['Half Bath']

In [ ]:
#another instance where data descr uses NA to mean no
train['Fireplace Qu'].fillna('NFP', inplace=True)

In [ ]:
train['Fireplaces*Fireplace Quality '] = train['Fireplaces']*train['Fireplace Qu']

In [ ]:
train.drop('Fireplaces',axis=1,inplace=True)
train.drop('Fireplace Qu', axis=1,inplace=True)

In [ ]:
#Na = no garage
train['Garage Type'].fillna('NG',inplace=True)

In [ ]:
train['Garage Yr Blt'].fillna(0,inplace=True)

In [ ]:
train['Garage Finish'].fillna('NG',inplace=True)

In [ ]:
train[train['Garage Cars'].isnull()].index

In [ ]:
#The null value in 'garage cars' corresponds to a instance where 'garage type' is full but other garage variables are also empty
#Therefore, I am dropping the row
train.drop(train.index[1712], inplace=True)

In [ ]:
train['Garage Qual'].fillna('NG', inplace=True)

In [ ]:
train['Garage Cond'].fillna('NG', inplace=True)

In [ ]:
#NAs in 'Pool Qual' correspond with 0's in Pool sq ft; therefore, filling as 'NP', no pool

In [ ]:
train['Pool QC'].fillna('NP', inplace=True)

In [ ]:
train['Fence'].fillna('NF', inplace=True)

In [ ]:
train['Misc Feature'].fillna('None', inplace=True)

In [ ]:
train['Mo/Yr Sold']= (train['Mo Sold'].map(str)).str.cat((train['Yr Sold'].map(str)), sep="-")


In [ ]:
train.drop('Mo Sold', axis=1, inplace=True)

In [ ]:
train.loc[1699]['Garage Yr Blt']
#Garage could not have been built in a year that has yet to occur; removing

In [ ]:
train.drop([1699], axis=0, inplace=True)

In [ ]:
#creating an interaction term of these two closely related variables
train['Overall Quality Score'] = train['Overall Qual']*train['Overall Cond']

In [ ]:
train['Exterior Covering'] = (train['Exterior 1st']).str.cat((train['Exterior 2nd']), sep='&')

In [ ]:
train['Exterior Quality Score'] = (train['Exter Qual'].replace(['Ex', 'Gd','TA','Fa','Po'], [5,4,3,2,1]))*(train['Exter Cond'].replace(['Ex', 'Gd','TA','Fa','Po'], [5,4,3,2,1]))

In [ ]:
train.drop('Exter Qual', axis=1, inplace=True)
train.drop('Exter Cond', axis=1,inplace=True)

In [ ]:
train['BsmtFin Quality Score'] = (train['BsmtFin Type 1'].replace(['GLQ','ALQ','BLQ','Rec','LwQ', 'Unf','NB'],[6,5,4,3,2,1,0]))*(train['BsmtFin Type 2'].replace(['GLQ','ALQ','BLQ','Rec','LwQ', 'Unf','NB'],[6,5,4,3,2,1,0]))

In [ ]:
train.drop('BsmtFin Type 1', axis=1,inplace=True)
train.drop('BsmtFin Type 2', axis=1, inplace=True)

# Plots to identify correlations and outliers & decide on course of action

In [ ]:
train.describe()

In [ ]:
plt.figure(figsize= (16,16))
sns.heatmap(train.corr(),cmap='RdBu');

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap((train.corr()[['SalePrice']]).sort_values('SalePrice', ascending=False), annot=True) #sourced from code written by Mahdi S.
plt.title('Sale Price Correlations');

#### From the above heatmaps, we can see which variables are most closely correlated with sale price and each other:
                                 * Overall Quality
                                 * Gr Liv Area (Above Ground Living Area SF)
                                 * Garage Cars/Garage Area
                                 * 1st Fl SF
                                 * Bsmt Total SF
                                 * Full Baths

With this information, I can identify other variables correlated with these to expand/play around with the variables of my models.

In [ ]:
sns.boxplot(train['Overall Qual'])
plt.title('Overall Quality Rankings', size=15)
plt.xlabel('Overall Quality Ranking');

The clear outliers here are properties with overall quality values below 2. <br>
There are only 4 instances of these super low scores. <br> Therefore, I think it is prudent to remove these as they may unnecessarily skew my model results.

In [ ]:
train['Overall Qual'].value_counts()

In [ ]:
train = train[train['Overall Qual'] != 1]

In [ ]:
sns.boxplot(train['Gr Liv Area'])
plt.title('Above Ground Living Area', size=15)
plt.xlabel('Above Ground Living Area (SqFt)');

In [ ]:
(train['Gr Liv Area']>=3000).value_counts()

Homes with an above ground living area square footage of over 3000sqft represent less than 1% of all homes. <br>
Rather than remove them, I will create a separate column for these 'larger area' homes.

In [ ]:
train['Larger Gr Liv Area'] = ((train['Gr Liv Area']>=3000)==True)

In [ ]:
sns.boxplot(train['Garage Cars'])
plt.title('Car Capacity of Garage', size=15)
plt.xlabel('Number of Cars');

The clear outliers here are garages with car capacities of 4 & 5. <br> There is only 1 garage that accomodates 5 cars, and it does not fall into the 'Larger Gr Living Area' column (as would be expected as ground living area sqft and garage car capacities are otherwise correlated columns). <br> 
We can conclude it is an anomaly and not of benefit to my models. It will be removed.

In [ ]:
train['Garage Cars'].value_counts()

In [ ]:
train = train[train['Garage Cars'] != 5]

In [ ]:
sns.boxplot(train['Garage Area'])
plt.title('Garage Area (SqFt)')
plt.xlabel('Garage Area (SqFt)');

Homes with garage areas over 1000 SqFt are rare, and only 1 contains a garage over 1400, an anomaly. <br> I will drop this row.

In [ ]:
(train['Garage Area']>1000).value_counts()

In [ ]:
(train[train['Garage Area']>1400]).index

In [ ]:
train.drop([960],inplace=True)

Due to the clear relationship between the 'Garage Area' & 'Garage Cars' columns, I will create an interaction term of the two.

In [ ]:
sns.boxplot(train['1st Flr SF'])
plt.title('1st Floor Total SqFt', size=15)
plt.xlabel('Square Feet');

Of these outlier home sizes, there is only one home with a square footage over 5000 , one home with a square footage over 4000 , and one with a square footage over 3000. <br> These I will remove from my dataset. <br>The homes with square footage over 2000 mostly also appear in the variable 'Larger Gr Liv Area' I created to represent homes with an overall sqft living area over the max 3000.

In [ ]:
(train['1st Flr SF']>2000).value_counts()

In [ ]:
(train[(train['1st Flr SF']>2000) & (train['Larger Gr Liv Area']==True)])

In [ ]:
train = train[train['1st Flr SF'] <3000]

In [ ]:
sns.boxplot(train['2nd Flr SF'])
plt.title('2nd Floor Total SqFt', size=15)
plt.xlabel('Square Feet')

There are 3 outliers here. Homes whose second floor square footage exceeds 1800. Because these 3 match up with homes in my category for larger than avg above ground living area, I will keep them.

In [ ]:
(train['2nd Flr SF']>1800).value_counts()

In [ ]:
(train[(train['2nd Flr SF']>1800) & (train['Larger Gr Liv Area']==True)])

In [ ]:
sns.boxplot(train['Bsmt Total SF'])
plt.title('Total Basement Area (Finished&Unfinished)', size=15)
plt.xlabel('Square Feet');

There are only 2 houses wherein the total SF of the basement is greater than 3000. <br>
I am creating a new column that uses 1 to denote if a house's finished square footage is greater than half of the total basement square footage (0 if not). <br>
My assumption is that because total basement sq footage is positively correlated with sale price, the larger the basement, the more the house is worth; the more finished square footage of the basement, the higher the sale price.

In [ ]:
LgFin = []
for index, row in train.iterrows():
    if (row['BsmtFin SF 1']+ row['BsmtFin SF 2'])>(row['Bsmt Total SF']/2):
        LgFin.append(1)
    else:
        LgFin.append(0)

In [ ]:
train['Over 50%Fin Bsmt'] = LgFin

In [ ]:
train['BsmtFin SF'] = train['BsmtFin SF 1'] + train['BsmtFin SF 2']

In [ ]:
train.drop(['PID'], axis=1, inplace=True)

In [ ]:
g = sns.boxplot(train['Fence'], train['SalePrice'])
flabels = ['No Fence', 'Minimum Wood/Wire','Good Wood','Minimum Privacy','Good Privacy']
g.set_xticklabels(flabels,rotation=45)
plt.title('Fence Type Rating', size=15)
plt.xlabel('Fence Type');

In [ ]:
#Because I would like to use fence - or at least determine if it is worth using - in my model, I am converting the categ. values into numbers
train['Fence'] = train['Fence'].replace(['NF','MnWw','GdWo','MnPrv','GdPrv'], [0,1,2,3,4])

#### My instinct is that, as with most places, neighborhood is a large factor in determining the value/"sell-ability" of a home. <br> Let's check this out.

In [ ]:
plt.figure(figsize=(12,12))
g = sns.pointplot(x=train['Neighborhood'], y=train['SalePrice'], data=train, color='red')
labels=['Sawyer','Sawyer West',
        'North Ames',
        'Timberland',
        'Edwards', 
        'Old Town', 
        'Briardale',
        'College Creek', 
        'Somerset',
        'Mitchell',
        'Stonebrook',
        'Northridge Heights',
        'Gilbert',
        'Crawford',
        'Iowa DOT & Railroad',
        'Northwest Ames',
        'Veenker',
        'Meadow Village',
        'S&W Iowa State U',
        'Northridge',
        'Clear Creek', 
        'Bloomington Heights', 
        'Brookside', 
        'Northpark Villa', 
        'Bluestem', 
        'Green Hills', 
        'Greens', 
        'Landmark']
g.set_xticklabels(labels,rotation=70)
plt.title('Sale Price by Neighborhood', size=15)
plt.ylabel('Price (USD)')
plt.xlabel('Neighborhood')
mean= train['SalePrice'].mean()
plt.axhline(mean, color='green', linestyle='--');

The above graph shows sale price of homes by the neighborhood in which they are located. The green line represents the average price of a home sold in Ames.

In [ ]:
ax = pd.pivot_table(train, index=['Neighborhood'], values=['SalePrice'], columns = ['Yr Sold']).plot(kind='bar', width = .9, figsize=(14,10))
ax.set_xlabel('Neighborhood')
ax.set_title('Prices of Homes Sold in Ames Neighborhoods by Year', fontsize=15)
ax.legend(labels=['2006','2007','2008','2009','2010']);

Above, the price of homes sold by neighborhood for each year the data is available.<br> 
Below, the number of homes sold by neighborhood for each year the data is available.

In [ ]:
ax = pd.pivot_table(train, index=['Neighborhood'], values=['SalePrice'], columns = ['Yr Sold'], aggfunc=[len]).plot(kind='bar', figsize=(14,10))
ax.set_xlabel('Neighborhood')
ax.set_title('Number of Homes Sold in Ames Neighborhoods by Year', fontsize=15)
ax.legend(labels=['2006','2007','2008','2009','2010']);

While more homes were sold in the North Ames neighborhood, they were not necessarily the highest sale prices. <br> The Northridge, Northridge Heights, Green Hills, and Stonebrook neighborhoods had the highest priced homes sold.<br>
Aside: Home sales in Green Hills were conspicuously low/absent for years after 2007 (recall the recession hit in 2008).

My interest in this variable has been piqued. In order to use it in my Linear Regression model, I am going to convert the data in numerical dummy columns.

In [ ]:
dummies = pd.get_dummies(train['Neighborhood'])

In [ ]:
train = pd.concat([train, dummies], axis=1)
train.head(1)

In [ ]:
train.drop('Neighborhood', axis=1,inplace=True)

#### Saving my cleaned data to a new csv

In [ ]:
train.to_csv('../datasets/cleaned_training_data.csv')

### A revised sale price corr heatmap with my cleaned data

In [ ]:

plt.figure(figsize=(13,20))
sns.heatmap((train.corr()[['SalePrice']]).sort_values('SalePrice', ascending=False), annot=True)
plt.title('Sale Price Correlations', size=15);

#### A revised total heatmap with my cleaned data

In [ ]:
plt.figure(figsize= (16,16))
sns.heatmap(train.corr(),cmap='RdBu');

Though this neighborhood transformation has added 28 new variables to my list of possible features, this only prompts more selectivity in what other variables will be used in my analysis modeling.

In [ ]:
#will run when I have more time; takes forever
#sns.pairplot(train)

In [ ]:
plt.bar(train['Overall Qual'], train['SalePrice'] , color='#E84855')
plt.ylabel('Sale Price (USD)')
plt.xlabel('Overall Quality (1-10)')
plt.title('Sale Price by Home Quality Rating', size=15)
mean = train['Overall Qual'].mean()
smean = train['SalePrice'].mean()
plt.axvline(mean, color='black', linestyle='--')
plt.axhline(smean, color = 'green', linestyle='--');

In [ ]:
plt.figure(figsize=(10,10))
plt.bar(train['Overall Quality Score'], train['SalePrice'] , color='#2B3A67')
plt.ylabel('Sale Price (USD)')
plt.xlabel('Home Quality + Condition Score')
plt.title('Sale Price by Quality Score')
mean = train['Overall Quality Score'].mean()
smean = train['SalePrice'].mean()
plt.axhline(smean, color = 'green', linestyle='--')
plt.axvline(mean, color='red', linestyle='--');

# Next Steps

### After cleaning my data and performing the necessary functions on variables, I am ready to begin creating and testing out a few models.

### I have chosen to:
                    * combine columns with similar or highly related variables
                    * eliminate outliers that appeared in quantities under 10
                    * keep outliers that appeared in quantities over 10 because eliminating too many outliers may cause one or multiple iterations of my model to be overfit later on
                    * create new columns for outlier variables I think will be significant in determining my target later
                    

In [ ]:
train.shape

In [ ]:
#reordering so that my target column is the last one in my dataframe

#cols = list(train.columns.values)
#cols.pop(cols.index('SalePrice')) 
#train = train[cols+['SalePrice']]

In [ ]:
#def scatterem(df, y):
    #for col in range (0,3):
        #fig, axs = plt.subplots()
        #axs.scatter(df.iloc[:,col],y)
    #return fig,ax

In [ ]:
#scatterem(train, train['SalePrice'])

In [ ]:
train['SalePrice'].mean()

In [ ]:
plt.figure(figsize=(10,8))
plt.hist(train['SalePrice'], color = '#9BD1E5')
plt.title('Sale Price Distribution', size=15)
plt.xlabel('Sale Price (USD)')
plt.xlabel('Frequency')
plt.axvline((train['SalePrice'].mean()), color='red', linestyle='--');

In [ ]:
plt.figure(figsize=(10,8))
ax = sns.distplot(train['SalePrice'], color='#8EF9F3', kde_kws={"color": "purple"})
ax.set_xlabel('Sale Price (USD)')
ax.set_title('Sale Price Histogram')
plt.show()

# Preprocessing

#### Feature Selection:
Considering my train dataset still contains 82 variables, utilizing all of them will make my model severely prone to overfitting.<br> My initial model will utilize a fraction of these, allowing for more precise tuning in later versions.

#### Feature Engineering
Before instantiating/fitting models, I will utilize scikit-learn's Train Test Split and Standard Scaler methods to prepare my data. <br>
Based on my [second heat map](#A-revised-total-heatmap-with-my-cleaned-data), the correlations between my independent variables were not so high that I felt there was significant multicollinearity to justify using scikit learn's Polynomial Features.

In [ ]:
ends = train[['Overall Quality Score', 'Year Built', 'Gr Liv Area','BsmtFin SF','Garage Area','Fence', 'Larger Gr Liv Area', 'Over 50%Fin Bsmt']].copy()

In [ ]:
train.drop(['Overall Quality Score', 'Year Built', 'Gr Liv Area', 'BsmtFin SF','Garage Area','Fence','Larger Gr Liv Area', 'Over 50%Fin Bsmt'], axis=1, inplace=True)

In [ ]:
train = pd.concat([train, ends], axis=1)

In [ ]:
train.head(2)

In [ ]:
X1 = train.iloc[:,72:108]
y = train['SalePrice']

In [ ]:
#sns.pairplot(train, x_vars =X1,y_vars=y)

In [ ]:
X1_train, X1_test, y_train, y_test = train_test_split(X1,y, random_state=42)

In [ ]:
ss= StandardScaler()
X1_train_sc = ss.fit_transform(X1_train)
X1_test_sc = ss.transform(X1_test)

# Finally.... Modeling

### I will be using 3 types of models:

#### Linear Regression

#### Ridge Regression

#### Lasso Regression

### Linear

In [ ]:
#Instantiate and fit model
lr = LinearRegression()

lr.fit(X1_train_sc, y_train)

In [ ]:
lr_scores = cross_val_score(lr, X1_train_sc, y_train, cv=3)
lr_scores.mean()

### Ridge

In [ ]:
ridge = RidgeCV(alphas=np.linspace(.1, 10, 10))

ridge.fit(X1_train_sc,y_train)

In [ ]:
ridge_scores = cross_val_score(ridge, X1_train_sc, y_train, cv=3)
ridge_scores.mean()

### Lasso

In [ ]:
lasso = LassoCV(n_alphas=10)

lasso.fit(X1_train_sc, y_train)

In [ ]:
lasso_scores = cross_val_score(lasso, X1_train_sc, y_train, cv=3)
lasso_scores.mean()

### Lasso seems to be performing the best, so I will stick with that

In [ ]:
preds = lasso.predict(X1_test_sc)

In [ ]:
residuals = y_test - preds

In [ ]:
plt.figure(figsize=(12,10))
plt.scatter(preds, residuals, c=preds)
plt.title('Residual Distribution', size=15)
plt.xlabel('Predicted Price')
plt.ylabel('Residual')
plt.axhline(0, color='black');

In [ ]:
plt.figure(figsize=(12,10))
plt.scatter(y_test,preds, c=preds)
plt.title('Actual V. Predicted', size=15)
plt.xlabel('Actual Price (USD)')
plt.ylabel('Predicted Price (USD)');

In [ ]:
lasso.coef_

In [ ]:
X1_train_sc

In [ ]:
lasso_coef = pd.Series(lasso.coef_, index = X1_train.columns)

In [ ]:
lasso_coef.plot(kind = "barh", color='violet', figsize=(8,16))
plt.title("Coefficients in the Lasso Model");

The above chart shows the coefficients of my chosen variables in my lasso model.
This allows me to determine what variables are most important to determining value of a home; those that add value, and those that cause a decline.
